# Data Preprocessing 🧹

##Cleaning the Data

###Handling  Missing Customer ID values.

In [2]:
# Let’s see what’s missing
print("Missing values:\n", df.isnull().sum())

# Dropping rows where CustomerID is blank since I need it
df = df.dropna(subset=['CustomerID'])

# Double-checking to see if it worked
print("After dropping missing CustomerID:\n", df.isnull().sum())

Missing values:
 InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
After dropping missing CustomerID:
 InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


dropping missing customerID's. As we drop null values in customer ID. the associated null calues in despriction will also become 0.

## Removing negative values in qunatity and prices.

In [3]:
#Removing negative or zero quantites and prices
df= df[(df['Quantity']> 0) & (df['UnitPrice']>0)]

# Verifying
print("Rows after filtering:", len(df))
print(df.describe())

Rows after filtering: 397884
            Quantity                    InvoiceDate      UnitPrice  \
count  397884.000000                         397884  397884.000000   
mean       12.988238  2011-07-10 23:41:23.511023360       3.116488   
min         1.000000            2010-12-01 08:26:00       0.001000   
25%         2.000000            2011-04-07 11:12:00       1.250000   
50%         6.000000            2011-07-31 14:39:00       1.950000   
75%        12.000000            2011-10-20 14:33:00       3.750000   
max     80995.000000            2011-12-09 12:50:00    8142.750000   
std       179.331775                            NaN      22.097877   

          CustomerID  
count  397884.000000  
mean    15294.423453  
min     12346.000000  
25%     13969.000000  
50%     15159.000000  
75%     16795.000000  
max     18287.000000  
std      1713.141560  


## Removoing duplicates

In [4]:
# Check for duplicates
print("Duplicate rows:", df.duplicated().sum())

# Removing duplicates
df = df.drop_duplicates()

# Verifying
print("Rows after removing duplicates:", len(df))

Duplicate rows: 5192
Rows after removing duplicates: 392692


## Calculating the total spend per tansaction for futhure use.

In [5]:
# Calculating TotalPrice
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Verifying
df[['Quantity', 'UnitPrice', 'TotalPrice']].head()

,Quantity,UnitPrice,TotalPrice
0,6,2.55,15.30
1,6,3.39,20.34
2,8,2.75,22.00
3,6,3.39,20.34
4,6,3.39,20.34


### InvoiceDate needs to be in datetime format.

In [8]:
# Converting InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Verifying
print(df.dtypes)

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
TotalPrice            float64
dtype: object


In [9]:
# Saving cleaned data
df.to_csv("data/cleaned_data.csv", index=False)
print("Cleaned data saved to data/cleaned_data.csv")

Cleaned data saved to data/cleaned_data.csv


## Featuring Engineering. (RFM)

In [10]:
# Loading cleaned data
df = pd.read_csv("data/cleaned_data.csv")
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])  # Reconvert datetime

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


## Calculating RFM Scores

Recency gives the days since the customers last purchase

frequency gives number of unique invoices(purchases) per customer.

Monetary gives total spend per customer.

In [11]:
# Need a date to compare against—picking one day after the last sale
from datetime import datetime
reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
print("Reference date:", reference_date)

# Grouping by CustomerID to get RFM
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,  # Days since last buy
    'InvoiceNo': 'nunique',                                    # How many purchases
    'TotalPrice': 'sum'                                        # Total money spent
}).rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalPrice': 'Monetary'
})

# Checking out the results
print(rfm.head())
print(rfm.describe())

Reference date: 2011-12-10 12:50:00
            Recency  Frequency  Monetary
CustomerID                              
12346.0         326          1  77183.60
12347.0           2          7   4310.00
12348.0          75          4   1797.24
12349.0          19          1   1757.55
12350.0         310          1    334.40
           Recency    Frequency       Monetary
count  4338.000000  4338.000000    4338.000000
mean     92.536422     4.272015    2048.688081
std     100.014169     7.697998    8985.230220
min       1.000000     1.000000       3.750000
25%      18.000000     1.000000     306.482500
50%      51.000000     2.000000     668.570000
75%     142.000000     5.000000    1660.597500
max     374.000000   209.000000  280206.020000


This gave me a table with each customer’s RFM scores—pretty cool to see!

In [12]:
# Saving my RFM data as a CSV
rfm.to_csv("data/rfm_data.csv", index=False)
print("RFM data saved to data/rfm_data.csv")

RFM data saved to data/rfm_data.csv


By the end of this phase, I turned that chaotic Excel file into a clean dataset and set up RFM scores. It took some effort for dropping rows, fixing numbers, and grouping stuff—but now I’ve got a solid base for exploring and modeling. Now ready for the next step.